In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from numba import njit, prange
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os

In [ ]:
@njit
def laplacian(Z, dx):
    Ztop = Z[0:-2, 1:-1]
    Zleft = Z[1:-1, 0:-2]
    Zbottom = Z[2:, 1:-1]
    Zright = Z[1:-1, 2:]
    Zcenter = Z[1:-1, 1:-1]
    dif2 = Ztop + Zleft + Zbottom + Zright - 4*Zcenter
    dif2 = dif2/dx**2
    return dif2

@njit
def laplacian2D(Z,dx):

    center  = Z[1:-1, 1:-1]
    
    w   = Z[0:-2, 1:-1]
    e   = Z[2:  , 1:-1]   
    n   = Z[1:-1, 2:  ]
    s   = Z[1:-1, 0:-2]
    
    ne  = Z[2:  , 2:  ]
    nw  = Z[0:-2, 2:  ] 
    se  = Z[2:, 0:-2]
    sw  = Z[0:-2, 0:-2]
    
    dif2 = (w+e+n+s)/6.0 + (ne+nw+se+sw)/12.0 
    dif2 = dif2 - center
    
    dif2 = dif2/dx**2 
       
    return dif2

@njit
def PBC(Z):
    Z[0,:]  = Z[-2,:]
    Z[-1,:] = Z[1,:]
    Z[:,0]  = Z[:,-2]
    Z[:,-1] = Z[:,1]
    Z[-2,:] = Z[0,:]
    Z[0,:]  = Z[-1,:]
    return Z
    
@njit    
def NMC(Z):
    Z[0,:]  = Z[1,:]
    Z[-1,:] = Z[-2,:]
    Z[:,0]  = Z[:,1]
    Z[:,-1] = Z[:,-2]
    return Z

@njit 
def HS(c):
    return c*(3*c**2-9*c+8)/((1-c))**3

In [ ]:
@njit(parallel=True)
def solve_dynamics(q11, q12, c, dx, T, dt, n, R, alpha, r):
    for i in prange(n):
        
        Tr_q2 = 2*(q11**2 + q12**2)
        aux = c + 0.5 * alpha * Tr_q2
        nabla2_aux = laplacian2D(aux,dx)
        nabla2_q11 = laplacian2D(q11, dx)
        nabla2_q12 = laplacian2D(q12, dx)
        
        q11c = q11[1:-1,1:-1]
        q12c = q12[1:-1,1:-1]
        Tr_q2c = Tr_q2[1:-1,1:-1]
        cc = c[1:-1,1:-1]
        
        q11[1:-1,1:-1] = q11c + dt * (nabla2_q11 - r*q11c - 0.5*q11c*(Tr_q2c) - cc*q11c)
        q12[1:-1,1:-1] = q12c + dt * (nabla2_q12 - r*q12c - 0.5*q12c*(Tr_q2c) - cc*q12c)
        c[1:-1,1:-1]   = cc + dt * R * nabla2_aux
        
        
        for x in (q11, q12, c):
             PBC(x)

In [ ]:
def save_fields(s, c, theta, step, nb):
    LC_file = "matrix" + "_s_" + str(step) + "_nb_" + str(nb) + ".txt"
    np.savetxt(LC_file, s, delimiter="\t")
    
    NPs_file = "matrix" + "_c_" + str(step) + "_nb_" + str(nb) + ".txt"
    np.savetxt(NPs_file, c, delimiter="\t")
    
    Theta_file = "matrix" + "_theta_" + str(step) + "_nb_" + str(nb) + ".txt"
    np.savetxt(Theta_file, theta, delimiter="\t")

In [ ]:
def grafica(s,c, Theta):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3,figsize=(15, 10))

    ax1.imshow(c)
    im1 = ax1.imshow(c, cmap=plt.cm.gray)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes('right', size='5%', pad=0.5)
    fig.colorbar(im1, cax=cax, orientation='vertical', cmap=plt.cm.gray)
    fig.tight_layout()

    ax2.imshow(s)
    im2 = ax2.imshow(s, cmap=plt.cm.seismic)
    divider = make_axes_locatable(ax2)
    cax = divider.append_axes('right', size='5%', pad=0.5)
    fig.colorbar(im2, cax=cax, orientation='vertical')

    ax3.imshow(theta)
    im3 = ax3.imshow(theta, cmap=plt.cm.twilight_shifted)
    divider = make_axes_locatable(ax3)
    cax = divider.append_axes('right', size='5%', pad=0.5)
    fig.colorbar(im3, cax=cax, orientation='vertical')
    plt.show()

In [ ]:
N = 128

dx = 1.0
T = 1
dt = 0.01
n = int(T/dt)
R = 10.0
alpha = 0.05
r = -1.0

nb = 1
#plt.imshow(c, cmap = plt.cm.gray)
#plt.show()

t0 = time.time()
for block in range(1, nb):
    q11 = np.random.uniform(-0.001, 0.001, (N,N))
    q12 = np.random.uniform(-0.001, 0.001, (N,N))
    c = np.random.uniform(0.009, 0.011,  (N,N))

    
    for x in (q11,q12,c):
        PBC(x)
    s = np.sqrt(q11**2 + q12**2)
    theta = np.arctan2(1.0, (q11 + s)/q12) 
    save_fields(s, c, theta, 0, block)
    for i in range(1,100):
        print('step: ', i)
        solve_dynamics(q11, q12, c, dx, T, dt, n, R, alpha, r)
        s     = np.sqrt(q11**2 + q12**2) 
        theta = np.arctan2(1.0, (q11 + s)/q12)
        #save_fields(s, c, theta, i, block)
        grafica(s,c,theta)
        
tf = time.time()


Deltat = tf-t0
print("Tiempo real (s) (min) (hrs): ", Deltat, Deltat/60, Deltat/3600)